# Prerequisites

In [ ]:
# path to the root directory from the Google Drive root
GD_DATA_ROOT_PATH = 'UoB Project/training_data'

# data directory name in the <GD_DATA_ROOT_PATH>
# NOTE: in the dir, there should be one zip file 
# that contain all the required data files including images to configuration
GD_DATA_DIR_NAME = 'd5_i321_c4_r416_sd64'

# Yolo v4 configuration data
SUBDIVISIONS = '64'
YOLO_WIDTH = '416'
YOLO_HEIGHT = '416'

In [ ]:
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)



# Building Darknet



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd /content/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
!make

In [ ]:
!ln -s /content/darknet/darknet /usr/bin

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
!darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg

In [ ]:
imShow('predictions.jpg')

# Setting up Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ln -s "/content/drive/My Drive/{GD_DATA_ROOT_PATH}" /data

In [ ]:
%cd /content/darknet
!unzip "/data/{GD_DATA_DIR_NAME}/*.zip" -d training
!cp cfg/yolov4.cfg training/obj.cfg

In [ ]:
%cd /content/darknet/training

n_classes = !echo -e $(sed -n /class/p obj.data | cut -d'=' -f2)
n_classes = int(n_classes[0])

print('No of classes::' + str(n_classes))

!sed -i 's/backup = backup\//backup = \/data\/{GD_DATA_DIR_NAME}\/backup\//' obj.data

!sed -i 's/subdivisions=8/subdivisions={SUBDIVISIONS}/' obj.cfg
!sed -i 's/width=608/width={YOLO_WIDTH}/' obj.cfg
!sed -i 's/height=608/height={YOLO_HEIGHT}/' obj.cfg

max_batches = n_classes * 2000
steps1 = int(max_batches / 100 * 80)
steps2 = int(max_batches / 100 * 90)
filters = (n_classes + 5) * 3

!sed -i 's/max_batches = 500500/max_batches = {max_batches}/' obj.cfg
!sed -i 's/steps=400000,450000/steps={steps1},{steps2}/' obj.cfg
!sed -i 's/classes=80/classes={n_classes}/' obj.cfg
!sed -i 's/filters=255/filters={filters}/' obj.cfg

In [ ]:
%cd /content/darknet
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
assert False

In [ ]:
import os.path

%cd /content/darknet

backup_path = '/data/{}/backup'.format(GD_DATA_DIR_NAME)
backup_weights_path = '{}/obj_last.weights'.format(backup_path)

backup_log_path = '{}/output.txt'.format(backup_path)
backup_cfg_path = '{}/obj.cfg'.format(backup_path)
backup_data_path = '{}/obj.data'.format(backup_path)
backup_names_path = '{}/obj.names'.format(backup_path)

config_path = 'training/obj.cfg'
data_path = 'training/obj.data'
names_path = 'training/obj.names'
weights_path = 'yolov4.conv.137'

# copy the required files to drive
!mkdir -p "{backup_path}"
!touch -a "{backup_log_path}"
!cp "{config_path}" "{backup_cfg_path}"
!cp "{data_path}" "{backup_data_path}"
!cp "{names_path}" "{backup_names_path}"

if os.path.isfile(backup_weights_path):
  print("Backup weight file found!")
  print("Starting from last weight!")
  !darknet detector train "{data_path}" "{config_path}" "{backup_weights_path}" -dont_show &>> "{backup_log_path}"
else:
  print("Backup weight NOT found!")
  print("Starting from scratch")
  !darknet detector train "{data_path}" "{config_path}" "{weights_path}" -dont_show &> "{backup_log_path}"

In [ ]:
!ls /data/d1_321_c4/backup/

In [ ]:
# show chart.png of how custom object detector did with training
imShow('chart.png')